In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3906078321304530241
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 535064248807050423
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5609422848
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6594387772000064226
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2927000206528719732
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
train = pd.read_csv('digits/train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x = train.drop(['label'], axis = 1)
x.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y = train['label']
y.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [6]:
x = x/255
x = x.values.reshape(-1, 28, 28, 1)
x.shape

(42000, 28, 28, 1)

In [7]:
y = to_categorical(y, num_classes = 10)
y.shape

(42000, 10)

In [8]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()

In [9]:
x_train = x_train/255
x_train = x_train.reshape(-1, 784)
x_val = x_val/255
x_val = x_val.reshape(-1, 784)

In [10]:
print(x_train.shape)
print(x_val.shape)

(60000, 784)
(10000, 784)


In [11]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)
print(x_train.shape)
print(x_val.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [12]:
y_train = to_categorical(y_train, num_classes = 10)
y_val = to_categorical(y_val, num_classes = 10)

In [13]:
x_train = np.concatenate((x_train, x))
y_train = np.concatenate((y_train, y))

In [14]:
print(x_train.shape)
print(x_val.shape)

(102000, 28, 28, 1)
(10000, 28, 28, 1)


In [15]:
model = Sequential([
            Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
            Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'),
            MaxPool2D(pool_size=(2,2)),
            Dropout(0.25),
    
            Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
            Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
            MaxPool2D(pool_size=(2,2), strides = (2,2)),
            Dropout(0.25),
    
            Flatten(),
            Dense(256, activation = "relu"),
            Dropout(0.5),
            Dense(10, activation = "softmax")
    ])

In [16]:
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [17]:
model.fit(x_train, y_train, epochs = 30, batch_size = 32, validation_data = (x_val, y_val), verbose = 1)

Train on 102000 samples, validate on 10000 samples
Epoch 1/30
102000/102000 [==============================] - 21s 210us/step - loss: 0.1306 - accuracy: 0.9607 - val_loss: 0.0280 - val_accuracy: 0.9901
Epoch 2/30
102000/102000 [==============================] - 21s 203us/step - loss: 0.0495 - accuracy: 0.9853 - val_loss: 0.0168 - val_accuracy: 0.9951
Epoch 3/30
102000/102000 [==============================] - 21s 206us/step - loss: 0.0371 - accuracy: 0.9888 - val_loss: 0.0176 - val_accuracy: 0.9944
Epoch 4/30
102000/102000 [==============================] - 21s 204us/step - loss: 0.0319 - accuracy: 0.9903 - val_loss: 0.0125 - val_accuracy: 0.9954
Epoch 5/30
102000/102000 [==============================] - 21s 205us/step - loss: 0.0292 - accuracy: 0.9914 - val_loss: 0.0143 - val_accuracy: 0.9959
Epoch 6/30
102000/102000 [==============================] - 21s 204us/step - loss: 0.0268 - accuracy: 0.9920 - val_loss: 0.0135 - val_accuracy: 0.9963
Epoch 7/30
102000/102000 [=================

In [18]:
model.save_weights('model.h5')

In [19]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)